In [1]:
import csv
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
DATA_PATH = '../data/'
NREL_PATH = '../data/NREL_raw.csv'
NREL_NUM_PATH = '../data/NREL_numerical.csv'

In [4]:
data_df = np.fromfile('../data/encoded_station.npy')

In [7]:
data_df.shape

(353572,)

In [71]:
dataset_df = pd.read_csv(
    DATA_PATH + 'Google_Map_review_data_AA_DTW.csv', sep=',', header=0)
sample_df = pd.read_csv(
    DATA_PATH + 'NREL_numerical.csv', sep=',', header=0, low_memory=False)
authors_list = dataset_df.author_name.unique().tolist()


In [81]:
count = 0
bad_count = 0
fake_ids = {}
id_map = []
fake_id_map = []
stations_list = []
curr_id = 0
author_map = []
authors = {}
curr_author_id = 0
for row in dataset_df.itertuples():
    # if (float(row.lat), float(row.lng)) not in coor_list:
    #     coor_list.add((float(row.lat), float(row.lng)))
    mask = (abs(sample_df.Latitude - row.lat) < 0.001) & (
        abs(sample_df.Longitude - row.lng) < 0.001)
    masked_nrel = sample_df[mask]
    if len(masked_nrel) > 0:
        real_id = masked_nrel.iloc[0]['ID']
        id_map.append(real_id)
        if real_id in fake_ids:
            fake_id_map.append(fake_ids[real_id])
        else:
            fake_ids[real_id] = curr_id
            fake_id_map.append(curr_id)
            curr_id += 1
        if row.author_name in authors:
            author_map.append(authors[row.author_name])
        else:
            authors[row.author_name] = curr_author_id
            author_map.append(curr_author_id)
            curr_author_id += 1
        stations_list.append(row.address)
    else:
        stations_list.append('')
        id_map.append(-1)
        fake_id_map.append(-1)
        author_map.append(-1)
        bad_count += 1
    count += 1 
print('Total:', count)
print('Good:', count - bad_count)  
print('Percentage:', (count - bad_count) / count)   

Total: 354
Good: 305
Percentage: 0.8615819209039548


In [82]:
len(id_map), len(fake_id_map), len(stations_list), len(author_map)

(354, 354, 354, 354)

In [67]:
print('Total unique stations:', len(set(stations_list)))
print('Total users:', len(authors_list))

Total unique stations: 90
Total users: 306


In [50]:
from sklearn.model_selection import train_test_split

In [83]:
X = []
y = []
for idx, row in dataset_df.iterrows():
    author_id = author_map[idx]
    station_id = fake_id_map[idx]
    rating = row.rating
    if station_id == -1:
        continue
    X.append([author_id, station_id])
    y.append(rating)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [84]:
with open(DATA_PATH + 'u.train', 'w') as f:
    for i in range(len(X_train)):
        f.write(f'{X_train[i][0]} {X_train[i][1]} {y_train[i]}\n')

with open(DATA_PATH + 'u.test', 'w') as f:
    for i in range(len(X_test)):
        f.write(f'{X_test[i][0]} {X_test[i][1]} {y_test[i]}\n')

with open(DATA_PATH + 'u.base', 'w') as f:
    for i in range(len(X)):
        f.write(f'{X[i][0]} {X[i][1]} {y[i]}\n')

In [17]:
id_map = []
stations_list = []
dataset_df = pd.read_csv(
    DATA_PATH + 'Google_Map_review_data_AA_DTW.csv', sep=',', header=0)
sample_df = pd.read_csv(
    DATA_PATH + 'NREL_numerical.csv', sep=',', header=0)
authors_list = dataset_df.author_name.unique()
stations_list = []
coor_list = set()
for row in dataset_df.itertuples():
    if (float(row.lat), float(row.lng)) not in coor_list:
        coor_list.add((float(row.lat), float(row.lng)))
        mask = (abs(sample_df.Latitude - row.lat) < 0.001) & (
            abs(sample_df.Longitude - row.lng) < 0.001)
        masked_nrel = sample_df[mask]
        if len(masked_nrel) > 0:
            id_map.append(masked_nrel.iloc[0]['ID'])
            stations_list.append(row.name)

/tmp/ipykernel_36792/3189738852.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pd.read_csv(


In [18]:
stations_list

['Electrify America Charging Station',
 'ChargePoint Charging Station',
 'EVgo Charging Station',
 'Shell Recharge Charging Station',
 'Tesla Supercharger',
 'ChargePoint Charging Station',
 'ChargePoint Charging Station',
 'Tesla Supercharger',
 'ChargePoint Charging Station',
 'Electric Vehicle Charging Station',
 'ChargePoint Charging Station',
 'Red E Charging Station',
 'ChargePoint Charging Station',
 'ChargePoint Charging Station',
 'ChargePoint Charging Station',
 'Tesla Supercharger',
 'Electric Vehicle Charging Station',
 'RED E Charging Station',
 'EVgo Charging Station',
 'Electrify America Charging Station',
 'ChargePoint Charging Station',
 'ChargePoint Charging Station',
 'ChargePoint Charging Station',
 'ChargePoint Charging Station',
 'EVgo Charging Station',
 'Tesla Supercharger',
 'Greenlots Charging Station',
 'EVgo Charging Station',
 'Electric Vehicle Charging Station',
 'EVgo Charging Station',
 'ChargePoint Charging Station',
 'ChargePoint Charging Station',
 'C

In [6]:
df2 = pd.read_csv(NREL_NUM_PATH, low_memory=False)

In [8]:
df = pd.read_csv(NREL_PATH, delimiter='\t', low_memory=False)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66020 entries, 0 to 66019
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           66020 non-null  int64  
 1   station_name                 66019 non-null  object 
 2   street_address               65974 non-null  object 
 3   intersection_directions      2345 non-null   object 
 4   city                         66006 non-null  object 
 5   state_name                   66009 non-null  object 
 6   zip                          66007 non-null  object 
 7   plus4                        0 non-null      float64
 8   station_phone                62370 non-null  object 
 9   access_days_time             58947 non-null  object 
 10  cards_accepted               6106 non-null   object 
 11  ev_level1_evse_num           669 non-null    float64
 12  ev_level2_evse_num           57260 non-null  float64
 13  ev_dc_fast_count

In [8]:
with open(NREL_PATH, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    columns = next(reader)
    while True:
        row = next(reader)
        if row[0] == '53413':
            break
    for i, item in enumerate(row):
        if item == '':
            row[i] = 'NULL'
        else:
            row[i] = row[i].replace('"', '\'')
    command = f'INSERT INTO nrel (' + ', '.join(columns) + \
        ') VALUES (' + ', '.join(row) + ')'
    print(command)

INSERT INTO nrel (id, station_name, street_address, intersection_directions, city, state_name, zip, plus4, station_phone, access_days_time, cards_accepted, ev_level1_evse_num, ev_level2_evse_num, ev_dc_fast_count, ev_other_info, ev_network, ev_network_web, geocode_status, latitude, longitude, date_last_confirmed, updated_at, owner_type_code, federal_agency_id, federal_agency_name, open_date, ev_connector_types, country, access_code, access_detail_code, federal_agency_code, facility_type, ev_pricing, ev_on_site_renewable_source, restricted_access, nps_unit_name, maximum_vehicle_class, ev_workplace_charging) VALUES (53413, Lewis & Clark Community College, 5800 Godfrey Rd, One station between Erickson Hall and the Nursing Building in the 'Cabin Lot.' The other station is between Riverbend Arena and the Math Building., Godfrey, IL, 62035, NULL, 618-468-2782, 24 hours daily; visit security desk after normal business hours to use, NULL, NULL, 2.0, NULL, NULL, Non-Networked, NULL, GPS, 38.950

In [83]:
def csv2sql_schema(filepath, table_name, delimiter=','):
    df = pd.read_csv(filepath, delimiter=delimiter)
    type_map = {
        'int64': 'INTEGER',
        'float64': 'FLOAT',
        'object': 'TEXT'
    }

    columns_sql = []
    columns_sql.append(f"  id INTEGER PRIMARY KEY AUTOINCREMENT,\n")
    for column, dtype in df.dtypes.items():
        if column == 'ID' or 'French' in column:
            continue
        column = column.lower().replace(' ', '_').replace('-', '_')
        if column == 'state':
            column = 'state_name'
        columns_sql.append(f"  {column} {type_map[dtype.name]},\n")
    
    command = f"CREATE TABLE {table_name} (\n{''.join(columns_sql)});"
    command = command[:-4] + '\n);'
    return command

In [86]:
txt = csv2sql_schema(NREL_PATH, 'nrel', '\t')

/tmp/ipykernel_10050/3190012416.py:2: DtypeWarning: Columns (6,20,31,46,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, delimiter=delimiter)


In [85]:
with open('../electrifind/schema.sql', 'a', encoding='utf-8') as f:
    f.write('\n\n')
    f.write(txt)
f.close()